# Chapter 25: Jenkins

While Chapter 24 established general pipeline orchestration patterns, this chapter examines Jenkins—the open-source automation server that has served as the foundation of CI/CD for over a decade. Despite the proliferation of cloud-native alternatives, Jenkins remains ubiquitous in enterprise environments due to its extensibility, self-hosted control, and vast plugin ecosystem.

Understanding Jenkins architecture, from master-agent topologies to Pipeline DSL nuances, is essential for maintaining legacy systems and for scenarios requiring air-gapped or highly customized CI infrastructure. This chapter bridges traditional CI concepts with modern containerized workflows, demonstrating how Jenkins adapts to Kubernetes-native operations.

## 25.1 Jenkins Architecture

Jenkins operates on a distributed master-agent (formerly master-slave) architecture that separates the control plane from execution environments, enabling scalable, heterogeneous build fleets.

### Master (Controller)

The Jenkins master serves as the central orchestrator:
- **Web UI**: HTTP server providing the configuration interface and build visualization
- **Job Configuration**: Stores pipeline definitions, credentials, and system configuration
- **Build Queue**: Schedules and dispatches work to agents
- **Artifact Storage**: Archives build outputs (though external storage is recommended)
- **Plugin Coordination**: Manages extensions and integrations

**Resource Requirements:**
- CPU: 2-4 cores minimum (more for heavy UI usage)
- RAM: 4-8 GB heap for JVM (scale with number of concurrent builds)
- Disk: SSD recommended for `$JENKINS_HOME` (metadata and logs)

### Agent (Node) Architecture

Agents execute the actual build workloads, connecting to the master via:
- **JNLP (Java Web Start)**: Agents initiate connection to master
- **SSH**: Master connects to agents via SSH
- **Inbound/Outbound**: WebSocket or HTTP-based connections for cloud environments

**Agent Types:**
1. **Permanent Agents**: Static VMs or physical machines, always connected
2. **Cloud Agents**: Dynamic provisioning via Kubernetes, EC2, Azure VMs, or Docker
3. **Ephemeral Agents**: Created per-build, destroyed immediately after

### Distributed Build Topology

```yaml
# Kubernetes-based Jenkins architecture
# Master runs as StatefulSet with persistent volume
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: jenkins-master
spec:
  serviceName: jenkins
  replicas: 1  # Single master (HA requires additional configuration)
  selector:
    matchLabels:
      app: jenkins-master
  template:
    metadata:
      labels:
        app: jenkins-master
    spec:
      containers:
      - name: jenkins
        image: jenkins/jenkins:lts-jdk17
        ports:
        - containerPort: 8080
          name: http
        - containerPort: 50000
          name: jnlp  # Agent connection port
        volumeMounts:
        - name: jenkins-home
          mountPath: /var/jenkins_home
        resources:
          requests:
            memory: "4Gi"
            cpu: "2000m"
          limits:
            memory: "8Gi"
            cpu: "4000m"
  volumeClaimTemplates:
  - metadata:
      name: jenkins-home
    spec:
      accessModes: ["ReadWriteOnce"]
      storageClassName: fast-ssd
      resources:
        requests:
          storage: 50Gi
```

## 25.2 Installation and Setup

Jenkins supports multiple deployment models, from traditional WAR files to containerized and Kubernetes-native installations.

### Docker Installation (Single Instance)

```bash
# Production-ready Docker run with persistent volume
docker run -d \
  --name jenkins \
  --user root \
  -p 8080:8080 \
  -p 50000:50000 \
  -v jenkins_home:/var/jenkins_home \
  -v /var/run/docker.sock:/var/run/docker.sock \  # Docker access (security consideration)
  -e JAVA_OPTS="-Djenkins.install.runSetupWizard=false \
                -Djenkins.CLI.disabled=true \
                -Dhudson.model.DirectoryBrowserSupport.CSP=\"default-src 'self'; script-src 'self' 'unsafe-inline'; style-src 'self' 'unsafe-inline';\"" \
  jenkins/jenkins:lts-jdk17

# Install plugins automatically via init scripts
docker exec jenkins \
  jenkins-plugin-cli --plugins \
    workflow-aggregator:590.v6a_d052e5a_a_b_5 \
    git:5.2.1 \
    configuration-as-code:1647.ve39ca_b_829b_41 \
    kubernetes:3937.vd7b_82db_e347b_ \
    docker-workflow:563.vd5d2e5c4007f \
    credentials-binding:642.v737c34dea_6c2 \
    prometheus:2.5.1
```

### Kubernetes Installation (Jenkins Operator)

For production Kubernetes environments, use the Jenkins Operator for automated lifecycle management:

```yaml
# jenkins-instance.yaml
apiVersion: jenkins.io/v1alpha2
kind: Jenkins
metadata:
  name: production-jenkins
  namespace: jenkins
spec:
  master:
    image: jenkins/jenkins:lts-jdk17
    imagePullPolicy: Always
    resources:
      limits:
        cpu: "4000m"
        memory: "8Gi"
      requests:
        cpu: "2000m"
        memory: "4Gi"
    securityContext:
      runAsUser: 1000
      fsGroup: 1000
    containers:
    - name: jenkins-master
      command:
      - /usr/bin/tini
      - --
      - /usr/local/bin/jenkins.sh
  seedJobs:
  - id: seed-job
    targets: "cicd/jobs/*.jenkins"
    description: "Seed jobs for CI/CD pipelines"
    repositoryBranch: main
    repositoryUrl: https://github.com/company/jenkins-config.git
  service:
    type: ClusterIP
    port: 8080
```

### Initial Security Hardening

Upon first boot, Jenkins exposes setup wizard—automate security configuration:

```groovy
// init.groovy.d/01-disable-setup-wizard.groovy (placed in JENKINS_HOME)
import jenkins.model.*
import hudson.security.*
import jenkins.security.s2m.AdminWhitelistRule

def instance = Jenkins.getInstance()

// Create admin user programmatically
def hudsonRealm = new HudsonPrivateSecurityRealm(false)
hudsonRealm.createAccount("admin", System.getenv("JENKINS_ADMIN_PASSWORD"))
instance.setSecurityRealm(hudsonRealm)

// Configure authorization strategy
def strategy = new ProjectMatrixAuthorizationStrategy()
strategy.add(Jenkins.ADMINISTER, "admin")
strategy.add(Jenkins.READ, "authenticated")
instance.setAuthorizationStrategy(strategy)

// Enable CSRF protection
instance.setCrumbIssuer(new hudson.security.csrf.DefaultCrumbIssuer(true))

// Disable CLI over Remoting
instance.getDescriptor("jenkins.CLI").get().setEnabled(false)

// Enable Agent-to-Master security
instance.getInjector().getInstance(AdminWhitelistRule.class).setMasterKillSwitch(false)

instance.save()
```

## 25.3 Jenkins Pipeline (Declarative vs. Scripted)

Jenkins Pipeline provides domain-specific language (DSL) for defining CI/CD workflows as code, stored in `Jenkinsfile` within repositories.

### Declarative Pipeline

Structured, opinionated syntax with built-in validation, ideal for standard CI/CD workflows:

```groovy
// Jenkinsfile (Declarative)
pipeline {
    agent {
        kubernetes {
            yaml """
apiVersion: v1
kind: Pod
spec:
  containers:
  - name: node
    image: node:20-alpine
    command: ['cat']
    tty: true
  - name: docker
    image: docker:24-dind
    securityContext:
      privileged: true
    volumeMounts:
    - name: docker-graph-storage
      mountPath: /var/lib/docker
  volumes:
  - name: docker-graph-storage
    emptyDir: {}
"""
        }
    }
    
    environment {
        DOCKER_REGISTRY = 'registry.company.com'
        IMAGE_NAME = 'myapp'
        GIT_COMMIT_SHORT = sh(script: 'git rev-parse --short HEAD', returnStdout: true).trim()
    }
    
    options {
        buildDiscarder(logRotator(numToKeepStr: '10', artifactNumToKeepStr: '5'))
        timeout(time: 30, unit: 'MINUTES')
        disableConcurrentBuilds()
        timestamps()
    }
    
    stages {
        stage('Checkout') {
            steps {
                checkout scm
                sh 'git log --oneline -5'
            }
        }
        
        stage('Install Dependencies') {
            steps {
                container('node') {
                    sh 'npm ci'
                }
            }
        }
        
        stage('Parallel Validation') {
            parallel {
                stage('Lint') {
                    steps {
                        container('node') {
                            sh 'npm run lint'
                        }
                    }
                }
                stage('Unit Tests') {
                    steps {
                        container('node') {
                            sh 'npm test -- --coverage'
                        }
                    }
                    post {
                        always {
                            publishTestResults testResultsPattern: 'junit.xml'
                            publishHTML([
                                allowMissing: false,
                                alwaysLinkToLastBuild: true,
                                keepAll: true,
                                reportDir: 'coverage',
                                reportFiles: 'index.html',
                                reportName: 'Coverage Report'
                            ])
                        }
                    }
                }
                stage('Security Scan') {
                    steps {
                        container('node') {
                            sh 'npm audit --audit-level=high'
                        }
                    }
                }
            }
        }
        
        stage('Build Container') {
            steps {
                container('docker') {
                    sh """
                        docker build \
                            --build-arg GIT_COMMIT=${env.GIT_COMMIT_SHORT} \
                            --build-arg BUILD_DATE=\$(date -u +'%Y-%m-%dT%H:%M:%SZ') \
                            -t ${env.DOCKER_REGISTRY}/${env.IMAGE_NAME}:${env.GIT_COMMIT_SHORT} \
                            -t ${env.DOCKER_REGISTRY}/${env.IMAGE_NAME}:latest \
                            .
                    """
                }
            }
        }
        
        stage('Push Image') {
            when {
                branch 'main'
            }
            steps {
                container('docker') {
                    withCredentials([usernamePassword(
                        credentialsId: 'docker-registry-credentials',
                        usernameVariable: 'REGISTRY_USER',
                        passwordVariable: 'REGISTRY_PASS'
                    )]) {
                        sh """
                            echo \$REGISTRY_PASS | docker login -u \$REGISTRY_USER --password-stdin ${env.DOCKER_REGISTRY}
                            docker push ${env.DOCKER_REGISTRY}/${env.IMAGE_NAME}:${env.GIT_COMMIT_SHORT}
                            docker push ${env.DOCKER_REGISTRY}/${env.IMAGE_NAME}:latest
                        """
                    }
                }
            }
        }
        
        stage('Deploy to Staging') {
            when {
                branch 'main'
            }
            steps {
                container('node') {
                    withKubeConfig([credentialsId: 'k8s-service-account']) {
                        sh """
                            helm upgrade --install ${env.IMAGE_NAME} ./chart \
                                --namespace staging \
                                --set image.tag=${env.GIT_COMMIT_SHORT} \
                                --wait --timeout 5m
                        """
                    }
                }
            }
        }
    }
    
    post {
        always {
            cleanWs()
        }
        failure {
            slackSend(
                color: 'danger',
                message: "Build failed: ${env.JOB_NAME} #${env.BUILD_NUMBER} (<${env.BUILD_URL}|Open>)"
            )
        }
        success {
            slackSend(
                color: 'good',
                message: "Build succeeded: ${env.JOB_NAME} #${env.BUILD_NUMBER}"
            )
        }
    }
}
```

### Scripted Pipeline

Groovy-based, flexible syntax for complex logic and conditional flows:

```groovy
// Jenkinsfile (Scripted)
node('docker-agent') {
    try {
        // Custom workspace to avoid path issues
        ws("/tmp/build-${env.BUILD_NUMBER}") {
            stage('Preparation') {
                checkout scm
                env.GIT_COMMIT_SHORT = sh(script: 'git rev-parse --short HEAD', returnStdout: true).trim()
            }
            
            // Dynamic stage generation based on changes
            def services = ['api', 'web', 'worker']
            def parallelStages = [:]
            
            services.each { service ->
                parallelStages["Build ${service}"] = {
                    stage("Build ${service}") {
                        dir(service) {
                            sh 'make build'
                        }
                    }
                }
            }
            
            parallel parallelStages
            
            // Conditional logic based on Git changes
            stage('Determine Impact') {
                def changedFiles = sh(
                    script: 'git diff --name-only HEAD~1',
                    returnStdout: true
                ).trim()
                
                if (changedFiles.contains('database/migrations/')) {
                    stage('Database Migration') {
                        timeout(time: 10, unit: 'MINUTES') {
                            input message: 'Deploy database migrations?', ok: 'Deploy'
                            sh 'make migrate'
                        }
                    }
                }
            }
            
            stage('Deploy') {
                if (env.BRANCH_NAME == 'main') {
                    lock('production-deployment') {
                        timeout(time: 30, unit: 'MINUTES') {
                            milestone(1)  // Prevent older builds from deploying
                            sh 'make deploy-production'
                        }
                    }
                }
            }
        }
    } catch (err) {
        currentBuild.result = 'FAILURE'
        throw err
    } finally {
        notifyBuildStatus()
        deleteDir()
    }
}

def notifyBuildStatus() {
    def color = currentBuild.result == 'SUCCESS' ? 'good' : 'danger'
    slackSend(color: color, message: "Build ${currentBuild.result}: ${env.JOB_NAME}")
}
```

### Comparison Matrix

| Feature | Declarative | Scripted |
|---------|-------------|----------|
| **Syntax** | Structured, opinionated | Groovy, flexible |
| **Validation** | Schema validation | Runtime validation |
| **Visualization** | Rich Blue Ocean support | Basic |
| **Complex Logic** | Limited (requires script blocks) | Full Groovy capabilities |
| **Error Handling** | Built-in post conditions | Manual try-catch |
| **Restartability** | Stage-level restart | Full pipeline restart only |
| **Learning Curve** | Lower | Higher |

## 25.4 Creating Your First Pipeline

Establishing a Jenkins pipeline involves connecting source control, defining the Jenkinsfile, and configuring triggers.

### Multibranch Pipeline

Automatically discovers branches and Pull Requests:

```groovy
// Jenkinsfile in repository root
pipeline {
    agent any
    
    triggers {
        // Poll SCM every 5 minutes (if webhooks unavailable)
        pollSCM('H/5 * * * *')
        
        // Or use webhooks for instant trigger
        // Configured in GitHub/GitLab repository settings
    }
    
    environment {
        // Per-branch environment variables
        DEPLOY_ENV = "${env.BRANCH_NAME == 'main' ? 'production' : 'staging'}"
    }
    
    stages {
        stage('Build') {
            steps {
                echo "Building for branch: ${env.BRANCH_NAME}"
                sh 'echo "Build steps here"'
            }
        }
    }
}
```

**Jenkins Configuration (Configuration as Code):**
```yaml
# casc-config.yaml
jobs:
  - script: >
      multibranchPipelineJob('myapp-ci') {
        branchSources {
          branchSource {
            source {
              github {
                id('myapp-github')
                repoOwner('company')
                repository('myapp')
                credentialsId('github-app-credentials')
                traits {
                  gitHubBranchDiscovery {
                    strategyId(1)  // Exclude branches that are also filed as PRs
                  }
                  gitHubPullRequestDiscovery {
                    strategyId(2)  // Merging with target branch
                  }
                  wipeWorkspaceTrait()
                }
              }
            }
          }
        }
        factory {
          workflowBranchProjectFactory {
            scriptPath('Jenkinsfile')
          }
        }
        triggers {
          periodicFolderTrigger {
            interval('2m')  // Scan for new branches every 2 minutes
          }
        }
      }
```

## 25.5 Docker Integration

Jenkins provides multiple patterns for building Docker images, each with distinct security and performance characteristics.

### Docker-in-Docker (DinD)

Privileged containers with nested Docker daemon:

```groovy
pipeline {
    agent {
        kubernetes {
            yaml """
apiVersion: v1
kind: Pod
spec:
  containers:
  - name: docker
    image: docker:24-dind
    securityContext:
      privileged: true
    env:
    - name: DOCKER_TLS_CERTDIR
      value: ""
"""
        }
    }
    stages {
        stage('Build') {
            steps {
                container('docker') {
                    sh """
                        docker info
                        docker build -t myapp:${env.BUILD_NUMBER} .
                    """
                }
            }
        }
    }
}
```

**Security Warning:** Privileged containers have full host access. Use only in isolated, ephemeral build nodes.

### Docker Socket Mounting (DooD)

Mount host Docker socket (less isolated but simpler):

```groovy
agent {
    kubernetes {
        yaml """
apiVersion: v1
kind: Pod
spec:
  containers:
  - name: docker
    image: docker:24
    volumeMounts:
    - name: docker-sock
      mountPath: /var/run/docker.sock
  volumes:
  - name: docker-sock
    hostPath:
      path: /var/run/docker.sock
"""
    }
}
```

### Kaniko (Rootless Building)

Google's tool for building images without privileged access:

```groovy
pipeline {
    agent {
        kubernetes {
            yaml """
apiVersion: v1
kind: Pod
spec:
  containers:
  - name: kaniko
    image: gcr.io/kaniko-project/executor:debug
    command:
    - /busybox/cat
    tty: true
    volumeMounts:
    - name: jenkins-docker-cfg
      mountPath: /kaniko/.docker
  volumes:
  - name: jenkins-docker-cfg
    projected:
      sources:
      - secret:
          name: regcred
          items:
          - key: .dockerconfigjson
            path: config.json
"""
        }
    }
    stages {
        stage('Build with Kaniko') {
            steps {
                container(name: 'kaniko', shell: '/busybox/sh') {
                    sh """
                        /kaniko/executor \
                            --context \$(pwd) \
                            --dockerfile Dockerfile \
                            --destination registry.company.com/myapp:${env.BUILD_NUMBER} \
                            --cache=true \
                            --cache-ttl=24h
                    """
                }
            }
        }
    }
}
```

## 25.6 Kubernetes Integration with Jenkins

The Kubernetes Plugin enables dynamic agent provisioning, allowing Jenkins to create Pods on-demand for each build.

### Dynamic Pod Templates

```groovy
pipeline {
    agent {
        kubernetes {
            label 'jenkins-agent'
            defaultContainer 'jnlp'  // Jenkins agent container
            
            yaml """
apiVersion: v1
kind: Pod
metadata:
  labels:
    jenkins: agent
spec:
  containers:
  - name: node
    image: node:20-alpine
    command: ['cat']
    tty: true
    resources:
      requests:
        memory: "512Mi"
        cpu: "500m"
      limits:
        memory: "2Gi"
        cpu: "2000m"
  
  - name: maven
    image: maven:3.9-eclipse-temurin-21
    command: ['cat']
    tty: true
    volumeMounts:
    - name: m2-cache
      mountPath: /root/.m2
  
  - name: kubectl
    image: bitnami/kubectl:latest
    command: ['cat']
    tty: true
  
  volumes:
  - name: m2-cache
    persistentVolumeClaim:
      claimName: maven-repo-cache
"""
        }
    }
    
    stages {
        stage('Build Backend') {
            steps {
                container('maven') {
                    sh 'mvn clean package -DskipTests'
                }
            }
        }
        
        stage('Build Frontend') {
            steps {
                container('node') {
                    sh 'npm ci && npm run build'
                }
            }
        }
        
        stage('Deploy') {
            steps {
                container('kubectl') {
                    withKubeConfig([credentialsId: 'k8s-creds']) {
                        sh 'kubectl apply -f k8s/'
                    }
                }
            }
        }
    }
}
```

### Jenkins Kubernetes Operator

Production deployment using the official operator:

```yaml
apiVersion: jenkins.io/v1alpha2
kind: Jenkins
metadata:
  name: jenkins
spec:
  configurationAsCode:
    configurations:
    - name: jenkins-casc-config
    secret:
      name: jenkins-casc-credentials
  master:
    containers:
    - name: jenkins-master
      image: jenkins/jenkins:lts-jdk17
      resources:
        limits:
          cpu: "4000m"
          memory: "8Gi"
  seedJobs:
  - id: jenkins-operator
    targets: "cicd/jobs/*.jenkins"
    description: "Jenkins Operator repository"
    repositoryBranch: master
    repositoryUrl: https://github.com/jenkinsci/kubernetes-operator.git
```

## 25.7 Shared Libraries

Shared Libraries enable code reuse across multiple pipelines, centralizing common patterns and reducing duplication.

### Library Structure

```
(shared-library repository)
├── vars/                    # Global variables/steps
│   ├── buildDocker.groovy
│   ├── notifySlack.groovy
│   └── runTests.groovy
├── src/                     # Groovy source classes
│   └── com/company/
│       ├── Utils.groovy
│       └── Constants.groovy
├── resources/               # Non-Groovy files
│   └── scripts/
│       └── health-check.sh
└── README.md
```

### Global Variables (vars/)

```groovy
// vars/buildDocker.groovy
def call(Map config) {
    def imageName = config.imageName ?: error("imageName required")
    def tag = config.tag ?: env.GIT_COMMIT_SHORT
    def dockerfile = config.dockerfile ?: 'Dockerfile'
    def context = config.context ?: '.'
    
    stage("Build Docker: ${imageName}") {
        container('docker') {
            sh """
                docker build \
                    -f ${dockerfile} \
                    -t ${imageName}:${tag} \
                    ${context}
            """
            
            if (config.push && env.BRANCH_NAME == 'main') {
                withDockerRegistry([credentialsId: 'docker-hub']) {
                    sh "docker push ${imageName}:${tag}"
                }
            }
        }
    }
    return "${imageName}:${tag}"
}
```

**Usage in Jenkinsfile:**
```groovy
@Library('company-shared-library') _

pipeline {
    agent any
    stages {
        stage('Build') {
            steps {
                script {
                    def image = buildDocker(
                        imageName: 'myapp',
                        tag: env.BUILD_NUMBER,
                        push: true
                    )
                    env.BUILT_IMAGE = image
                }
            }
        }
    }
}
```

### Groovy Classes (src/)

```groovy
// src/com/company/Utils.groovy
package com.company

class Utils implements Serializable {
    def steps
    
    Utils(steps) {
        this.steps = steps
    }
    
    def isReleaseBranch(String branchName) {
        return branchName ==~ /^release\/\d+\.\d+$/
    }
    
    def generateSemanticVersion() {
        def tag = steps.sh(
            script: 'git describe --tags --always --first-parent',
            returnStdout: true
        ).trim()
        
        if (tag.startsWith('v')) {
            return tag.substring(1)
        }
        return "0.0.0-${steps.env.BUILD_NUMBER}"
    }
    
    def withAwsCredentials(Closure body) {
        steps.withCredentials([
            steps.string(credentialsId: 'aws-access-key', variable: 'AWS_ACCESS_KEY_ID'),
            steps.string(credentialsId: 'aws-secret-key', variable: 'AWS_SECRET_ACCESS_KEY')
        ]) {
            body()
        }
    }
}
```

**Usage:**
```groovy
@Library('company-shared-library') import com.company.Utils

def utils = new Utils(this)

pipeline {
    agent any
    stages {
        stage('Version') {
            steps {
                script {
                    if (utils.isReleaseBranch(env.BRANCH_NAME)) {
                        env.VERSION = utils.generateSemanticVersion()
                    }
                }
            }
        }
        stage('Deploy') {
            steps {
                script {
                    utils.withAwsCredentials {
                        sh 'aws s3 cp artifact.zip s3://bucket/'
                    }
                }
            }
        }
    }
}
```

## 25.8 Plugins and Extensions

Jenkins functionality extends through plugins—over 1800 available in the Update Center. Strategic plugin management is critical for stability.

### Essential Plugins for CI/CD

**Pipeline & Workflow:**
- `workflow-aggregator` (Pipeline plugin suite)
- `pipeline-stage-view` (Visual pipeline visualization)
- `pipeline-github-lib` (GitHub shared libraries)
- `pipeline-utility-steps` (File operations, JSON reading)

**Kubernetes & Docker:**
- `kubernetes` (Kubernetes plugin)
- `docker-workflow` (Docker Pipeline integration)
- `docker-build-step` (Docker build/push steps)

**Security:**
- `credentials-binding` (Inject credentials into builds)
- `matrix-auth` (Granular permissions)
- `role-strategy` (Role-based access control)
- `audit-trail` (Track configuration changes)

**Notifications:**
- `slack` (Slack integration)
- `email-ext` (Extended email notifications)
- `htmlpublisher` (Publish HTML reports)

**Code Quality:**
- `sonar` (SonarQube integration)
- `warnings-ng` (Aggregate compiler warnings)
- `cobertura` (Coverage reports)

### Plugin Version Management

Pin versions to prevent breaking changes:

```bash
# plugins.txt for Jenkins Docker image
workflow-aggregator:590.v6a_d052e5a_a_b_5
git:5.2.1
kubernetes:3937.vd7b_82db_e347b_
credentials-binding:642.v737c34dea_6c2
docker-workflow:563.vd5d2e5c4007f
blue-ocean:1.27.11
prometheus:2.5.1
```

**Installation via CLI:**
```bash
jenkins-plugin-cli --plugin-file plugins.txt --verbose
```

### Plugin Security

Regularly audit plugins for CVEs:
```bash
# Check for security warnings
curl -sSL "http://$JENKINS_URL/pluginManager/api/xml?depth=1&xpath=/*/*/shortName|/*/*/version&wrapper=plugins" | \
  perl -pe 's/.*?<shortName>([\w-]+).*?<version>([^<]+)()<\/version>/$1:$2\n/g' | \
  grep -v '^plugins$'
```

## 25.9 Security Configuration

Jenkins security requires hardening at multiple layers: authentication, authorization, build isolation, and secret management.

### Matrix-Based Security

```groovy
// init.groovy.d/03-security.groovy
import jenkins.model.*
import hudson.security.*
import hudson.security.csrf.DefaultCrumbIssuer

def instance = Jenkins.getInstance()

// Enable security
def realm = new HudsonPrivateSecurityRealm(false)
instance.setSecurityRealm(realm)

// Matrix authorization
def strategy = new ProjectMatrixAuthorizationStrategy()

// Admin group full access
strategy.add(Jenkins.ADMINISTER, "jenkins-admins")

// Developers can build and configure jobs
strategy.add(Jenkins.READ, "jenkins-developers")
strategy.add(Item.BUILD, "jenkins-developers")
strategy.add(Item.CANCEL, "jenkins-developers")
strategy.add(Item.CONFIGURE, "jenkins-developers")
strategy.add(Item.CREATE, "jenkins-developers")
strategy.add(Item.DELETE, "jenkins-developers")
strategy.add(Item.READ, "jenkins-developers")
strategy.add(View.READ, "jenkins-developers")

// CI service accounts (read-only)
strategy.add(Item.READ, "ci-service-account")
strategy.add(Item.BUILD, "ci-service-account")

instance.setAuthorizationStrategy(strategy)

// CSRF Protection
instance.setCrumbIssuer(new DefaultCrumbIssuer(true))

// Agent protocols (disable deprecated)
instance.setAgentProtocols(["JNLP4-connect", "Ping"])

instance.save()
```

### Credentials Management

Store credentials in Jenkins Credentials Provider, never in code:

```groovy
pipeline {
    agent any
    
    environment {
        // Inject credentials as environment variables
        DB_CREDENTIALS = credentials('production-database')
    }
    
    stages {
        stage('Deploy') {
            steps {
                // DB_CREDENTIALS_USR and DB_CREDENTIALS_PSW auto-injected
                sh '''
                    echo "Connecting as $DB_CREDENTIALS_USR"
                    psql -U $DB_CREDENTIALS_USR -p $DB_CREDENTIALS_PSW -h db.company.com
                '''
            }
        }
        
        stage('Alternative: WithCredentials') {
            steps {
                withCredentials([
                    usernamePassword(
                        credentialsId: 'docker-hub',
                        usernameVariable: 'DOCKER_USER',
                        passwordVariable: 'DOCKER_PASS'
                    ),
                    string(
                        credentialsId: 'api-key',
                        variable: 'API_KEY'
                    ),
                    file(
                        credentialsId: 'kubeconfig',
                        variable: 'KUBECONFIG'
                    )
                ]) {
                    sh '''
                        echo $DOCKER_PASS | docker login -u $DOCKER_USER --password-stdin
                        kubectl --kubeconfig=$KUBECONFIG apply -f deployment.yaml
                    '''
                }
            }
        }
    }
}
```

### Script Security Approval

Scripted Pipelines and shared libraries require approval of dangerous methods:

```groovy
// Manage Jenkins → In-process Script Approval
// Methods requiring approval:
// - java.lang.Runtime exec
// - java.io.File delete
// - Groovy JSON parsing (if using non-sandboxed libraries)

// Use @NonCPS for complex Groovy methods to avoid serialization issues
@NonCPS
def parseJson(String json) {
    new groovy.json.JsonSlurper().parseText(json)
}
```

## 25.10 Best Practices

### Pipeline as Code

Store all configuration in version control:

```bash
# Repository structure
jenkins/
├── Jenkinsfile              # Main pipeline
├── vars/                    # Shared library vars
├── resources/               # Config files
└── init.groovy.d/           # Bootstrap scripts (for Docker image)
```

### Backup and Disaster Recovery

```bash
# Backup Jenkins home (excluding workspace)
tar -czf jenkins-backup-$(date +%Y%m%d).tar.gz \
  --exclude='./workspace' \
  --exclude='./jobs/*/builds' \
  /var/jenkins_home

# Or use Configuration as Code plugin to export config
curl -X POST 'http://jenkins:8080/configuration-as-code/export' \
  -H 'Content-Type: application/json' \
  -d '{}' > jenkins-casc-backup.yaml
```

### Monitoring and Observability

```groovy
// Expose metrics for Prometheus
// Install prometheus plugin
pipeline {
    agent any
    
    options {
        // Add build metrics
        buildDiscarder(logRotator(numToKeepStr: '50'))
    }
    
    post {
        always {
            // Custom metrics
            prometheus metricName: 'build_duration_seconds', 
                      value: currentBuild.duration / 1000, 
                      labels: [job: env.JOB_NAME, branch: env.BRANCH_NAME]
        }
    }
}
```

### Agent Management Best Practices

1. **Immutable Agents**: Treat agents as cattle, not pets. Use containerized or cloud agents.
2. **Ephemeral Workspaces**: Clean workspace after each build (`cleanWs()`)
3. **Resource Limits**: Set CPU/memory limits on Kubernetes pods to prevent resource exhaustion
4. **Labeling Strategy**: Use meaningful labels (`docker`, `linux`, `high-memory`) for agent selection

---

## Chapter Summary and Preview

In this chapter, we examined Jenkins as a foundational CI/CD platform, exploring its distributed master-agent architecture that separates orchestration from execution. We detailed the installation options ranging from Docker containers to Kubernetes Operator deployments, emphasizing security hardening through scripted initialization and CSRF protection. The comparison between Declarative and Scripted Pipeline DSLs highlighted the trade-off between structured simplicity and flexible programmability, with Declarative preferred for standard workflows and Scripted for complex conditional logic. We implemented Docker integration through multiple patterns—Docker-in-Docker, socket mounting, and rootless Kaniko—each with distinct security postures. The Kubernetes Plugin demonstrated dynamic agent provisioning, enabling elastic build capacity that scales with demand while minimizing idle resource consumption. Shared Libraries emerged as critical for organizational standardization, allowing reusable pipeline components that embed security scanning and compliance checks consistently across repositories. Plugin management strategies emphasized version pinning and security auditing to prevent supply chain vulnerabilities. Finally, we established security best practices including matrix-based authorization, credential injection patterns that prevent secret exposure in logs, and the principle of treating build agents as ephemeral, immutable infrastructure.

**Key Takeaways:**
- Prefer Declarative Pipeline syntax for standard CI/CD workflows due to its validation, restartability, and rich visualization support; reserve Scripted Pipeline for complex logic requiring dynamic stage generation or sophisticated conditional flows
- Use Kaniko or Buildah for container image building in Jenkins to avoid privileged Docker containers, eliminating the security risks associated with Docker-in-Docker patterns while maintaining build isolation
- Implement Shared Libraries (vars/ for simple steps, src/ for complex classes) to centralize common pipeline patterns, ensuring that security scanning, notification, and deployment logic remain consistent and maintainable across hundreds of microservices
- Store Jenkins configuration as code (Jenkinsfile, CasC YAML, init.groovy.d scripts) in version control to enable disaster recovery, configuration auditing, and environment parity between Jenkins instances
- Treat Kubernetes-based Jenkins agents as ephemeral compute—provision per-build, enforce resource limits, and implement aggressive workspace cleanup to prevent disk exhaustion and cross-build contamination

**Next Chapter Preview:**
Chapter 26: GitHub Actions explores the cloud-native alternative to self-hosted Jenkins, examining event-driven workflow triggers, reusable composite actions, and the GitHub-hosted runner ecosystem. We will analyze how GitHub Actions integrates natively with pull requests, issues, and GitHub Packages, contrasting its serverless architecture with Jenkins' persistent infrastructure model. This chapter covers self-hosted runners for specialized workloads, matrix strategies across GitHub's hosted runner fleet, and security considerations including OIDC token authentication for cloud providers, equipping readers to choose between managed and self-hosted CI strategies based on organizational requirements.